In [13]:
import requests as rq
import pandas as pd
from datetime import datetime

def get_current_date_str():
    # Obtenir la date actuelle
    current_date = datetime.now()
    
    # Formater la date en chaîne de caractères au format jj-mm-yyyy
    date_str = current_date.strftime("%d-%m-%Y")
    
    return date_str

resp = rq.get(url= 'https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/api-velo-toulouse-temps-reel/records?limit=100').json()
resp 

{'total_count': 349,
 'results': [{'number': 253,
   'name': '00253 - ETATS-UNIS - ROBERT',
   'address': 'FACE 74 AV DES ETATS-UNIS (GARAGE RENAULT TOULOUSE)',
   'position': {'lon': 1.43234, 'lat': 43.630519},
   'banking': 'False',
   'bonus': 'False',
   'status': 'OPEN',
   'contract_name': 'toulouse',
   'bike_stands': 20,
   'available_bike_stands': 19,
   'available_bikes': 1,
   'last_update': '2024-07-30T17:57:02+00:00'},
  {'number': 330,
   'name': '00330 - LAPUJADE - BARTOLI',
   'address': 'Face 72, chemin de  Lapujade',
   'position': {'lon': 1.453553, 'lat': 43.622099},
   'banking': 'False',
   'bonus': 'False',
   'status': 'OPEN',
   'contract_name': 'toulouse',
   'bike_stands': 13,
   'available_bike_stands': 13,
   'available_bikes': 0,
   'last_update': '2024-07-30T10:52:28+00:00'},
  {'number': 366,
   'name': '00366 - ONCOPOLE TÉLÉO',
   'address': 'avenue Irène Joliot Curie',
   'position': {'lon': 1.428013, 'lat': 43.554843},
   'banking': 'False',
   'bonus'

In [6]:
utc = pytz.UTC
datetime.now(utc)

datetime.datetime(2024, 7, 30, 18, 20, 50, 434747, tzinfo=<UTC>)

In [25]:
from datetime import datetime, timedelta
from dateutil import parser
import pytz


def is_date_recent(date):
# Exemple de date reçue de l'API

    # Parser la chaîne de caractères en un objet datetime
    last_update = parser.parse(date)

    # Obtenir l'heure actuelle en UTC en utilisant pytz
    utc = pytz.UTC
    now = datetime.now(utc)

    # Calculer le delta de 5 minutes
    five_minutes_ago = now - timedelta(minutes=5)

    # Vérifier si 'last_update' est antérieur ou postérieur à il y a 5 minutes
    return last_update < five_minutes_ago


for rep in resp['results']:
    # print(rep)
    if not is_date_recent(rep['last_update']):
        print(rep['last_update'])
    


2024-07-30T17:57:02+00:00
2024-07-30T10:52:28+00:00
2024-07-30T09:36:38+00:00
2024-07-30T08:12:08+00:00
2024-07-30T04:06:21+00:00
2024-07-30T03:58:18+00:00
2024-07-29T09:43:55+00:00
2024-07-29T07:33:33+00:00
2024-07-26T08:48:04+00:00
2024-07-26T05:57:49+00:00
2024-07-25T15:23:05+00:00
2024-07-25T11:38:51+00:00
2024-07-25T05:39:34+00:00
2024-07-24T12:49:34+00:00
2024-07-24T11:27:12+00:00
2024-07-24T10:50:24+00:00
2024-07-24T09:07:05+00:00
2024-07-23T07:15:02+00:00
2024-07-20T10:07:28+00:00
2024-07-20T07:34:02+00:00
2024-07-15T13:39:22+00:00
2024-07-15T08:38:35+00:00
2024-07-10T07:55:33+00:00
2024-07-05T07:26:23+00:00
2024-06-25T11:43:48+00:00
2024-06-10T08:38:49+00:00
2024-02-19T06:19:52+00:00
2024-02-13T07:56:22+00:00
2023-09-25T05:52:16+00:00
2024-07-30T17:54:12+00:00
2024-07-30T17:50:55+00:00
2024-07-30T17:57:48+00:00
2024-07-30T17:53:26+00:00
2024-07-30T17:58:10+00:00
2024-07-30T17:56:37+00:00
2024-07-30T17:58:56+00:00
2024-07-30T17:57:39+00:00
2024-07-30T17:55:58+00:00
2024-07-30T1

In [53]:

def generate_number_to_name():
    
    station_names = []
    station_addresses = []
    station_numbers = []
    station_pos = []
    station_capacities = []
    status = []

    offset = 0
    limit = 100
    url = f'https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/api-velo-toulouse-temps-reel/records'
    total_count = rq.get(url=url).json()['total_count']
    
    while total_count > 0:
        if total_count < 100:
            limit = total_count%100
        
        url = f'https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/api-velo-toulouse-temps-reel/records?limit={limit}&offset={offset}'
        content = rq.get(url=url).json()['results']

        for station_resp in content:
            station_names.append(station_resp.get('name')[8:])
            station_addresses.append(station_resp.get('address'))
            station_pos.append((station_resp.get('position')['lon'], station_resp.get('position')['lat']))
            station_capacities.append(station_resp.get('bike_stands'))
            station_numbers.append(station_resp.get('number'))
            status.append(station_resp.get('status'))
        total_count -= 100
        offset +=100

    data = {'station_numbers': station_numbers,
            'station_names': station_names,
            'station_addresses': station_addresses,
            'station_pos': station_pos,
            'station_capacities': station_capacities,
            'status': status
            }
    
    df = pd.DataFrame(data).sort_values(by='station_numbers')
    current_date = datetime.now()
    df.to_csv(f'{current_date.strftime("%d-%m-%Y")}.csv', index=False)
    

generate_number_to_name()



In [20]:
url = f'https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/api-velo-toulouse-temps-reel/records?limit=100&offset=0'
rq.get(url=url).json()['results'][1]

{'number': 52,
 'name': '00052 - DUPORTAL - CITE ADMIN',
 'address': '39 BD ARMAND DUPORTAL',
 'position': {'lon': 1.434762, 'lat': 43.608571},
 'banking': 'False',
 'bonus': 'False',
 'status': 'OPEN',
 'contract_name': 'toulouse',
 'bike_stands': 21,
 'available_bike_stands': 20,
 'available_bikes': 0,
 'last_update': '2024-07-25T05:39:34+00:00'}